# About
Compare performance including and excluding hate against hegemonic groups using a logistic regression classifier.

Could merge with bert_hate_speech_classifier but probably not

# Train and evaluate LR hate speech classifiers
Load data, extract features, train classifiers, evaluate

In [23]:
# Load data
import pickle
import pdb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
from IPython.display import display

datasets = [
    'kennedy2020',
    'elsherief2021',
    'salminen2018',
    'sbic',
    'cad',
]

scores = {}
f1_scores = [] # List of dicts with keys: dataset, split, f1 (to create df)

for dataset in datasets[4:5]:
    print(dataset)
    
    path = f'/storage2/mamille3/hegemonic_hate/tmp/{dataset}_hegsplits_0.3hate.pkl'
    with open(path, 'rb') as f:
        data = pickle.load(f)
    if 'text' not in data[split]['train'].columns:
        pdb.set_trace()

    # Extract features
    print('Extracting features...')
    vectorizer = {}
    bow = {}
    for split in ['with_heg', 'no_heg']:
        # Check for NaNs
        if data[split]['train']['text'].isnull().values.any():
            pdb.set_trace()
        if data[split]['test']['text'].isnull().values.any():
            pdb.set_trace()
        vectorizer[split] = TfidfVectorizer(min_df=1)
        vectorizer[split].fit(data[split]['train']['text']) # corpus is a list of strings (documents)
        bow[split] = {}
        bow[split]['train'] = vectorizer[split].transform(data[split]['train']['text'])
        bow[split]['test'] = vectorizer[split].transform(data[split]['test']['text'])
    bow.keys()

    # Train LR model 
    print('Training model...')
    clf = {}
    for split in ['with_heg', 'no_heg']:
        clf[split] = LogisticRegression(solver='liblinear')
        clf[split].fit(bow[split]['train'], data[split]['train']['hate'])

    # Evaluate
    print('Evaluating...')
    scores[dataset] = {}
    for split in ['with_heg', 'no_heg']:
        preds = clf[split].predict(bow[split]['test'])
        true = data[split]['test']['hate']
        scores[dataset][split] = pd.DataFrame(classification_report(true, preds, output_dict=True))
        print(split)
        display(scores[dataset][split])

cad
Extracting features...
Training model...
Evaluating...
with_heg


,False,True,accuracy,macro avg,weighted avg
precision,0.787926,0.721631,0.778071,0.754778,0.768845
recall,0.941895,0.372711,0.778071,0.657303,0.778071
f1-score,0.858058,0.491546,0.778071,0.674802,0.752567
support,2702.000000,1092.000000,0.778071,3794.000000,3794.000000


no_heg


,False,True,accuracy,macro avg,weighted avg
precision,0.785714,0.752896,0.781234,0.769305,0.776268
recall,0.952628,0.357143,0.781234,0.654885,0.781234
f1-score,0.861158,0.484472,0.781234,0.672815,0.752739
support,2702.000000,1092.000000,0.781234,3794.000000,3794.000000
